In [1]:
import datetime

import numpy as np
import pandas as pd

import tensorflow as tf

2024-07-12 18:19:29.313225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 18:19:29.313282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 18:19:29.369381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 18:19:29.410819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.list_physical_devices('GPU')

2024-07-12 18:19:31.875046: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 18:19:32.046943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 18:19:32.047009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
EPOCHS = 100
BATCH_SIZE = 256

## Load Dataset

In [4]:
arr_train = np.loadtxt("../dataset/combined/train.csv", skiprows=1, delimiter=",")
arr_test = np.loadtxt("../dataset/combined/test.csv", skiprows=1, delimiter=",")
arr_val = np.loadtxt("../dataset/combined/validation.csv", skiprows=1, delimiter=",")

In [5]:
ds_train = tf.data.Dataset.from_tensor_slices((arr_train[:, :-1], arr_train[:, -1])).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = tf.data.Dataset.from_tensor_slices((arr_val[:, :-1], arr_val[:, -1])).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = tf.data.Dataset.from_tensor_slices((arr_test[:, :-1], arr_test[:, -1])).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2024-07-11 21:36:18.870642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 21:36:18.870902: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 21:36:18.870953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 21:36:19.252398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-11 21:36:19.252505: I external/local_xla/xla/stream_executor

## Model

In [4]:
def resblock(x, size):
  fx = tf.keras.layers.Dense(size, activation='relu')(x)
  fx = tf.keras.layers.BatchNormalization()(fx)
  fx = tf.keras.layers.Dense(size, activation='relu')(fx)
  out = tf.keras.layers.Add()([x, fx])
  out = tf.keras.layers.ReLU()(out)
  out = tf.keras.layers.BatchNormalization()(out)
  
  return out

In [8]:
def create_model(kind):
  # create input layer
  inputs = tf.keras.Input(shape=(42,))

  # create preprocessing layer
  norm_layer = tf.keras.layers.Normalization()
  # norm_layer.adapt(ds_train.map(lambda x, _: x))

  x = norm_layer(inputs)
  # x = inputs

  # create hidden layers
  if kind == 2:
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
  elif kind == 3:
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
  elif kind == 4 or kind == 5:
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = resblock(x, 128)
    x = resblock(x, 128)
    x = resblock(x, 128)

    x = resblock(x, 128)
    x = resblock(x, 128)
    x = resblock(x, 128)
    if kind == 5:
      x = resblock(x, 128)
      x = resblock(x, 128)
      x = resblock(x, 128)

      x = resblock(x, 128)
      x = resblock(x, 128)
      x = resblock(x, 128)

  # create output layer
  outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

  # create model
  return tf.keras.Model(inputs=inputs, outputs=outputs, name=f"model_{kind}")

In [9]:
model = create_model(4)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryFocalCrossentropy(gamma=2, from_logits=False),
    metrics=[
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
])

model.summary()

2024-07-12 18:20:07.482163: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 18:20:07.482375: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 18:20:07.482435: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 18:20:07.674462: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-12 18:20:07.674951: I external/local_xla/xla/stream_executor

Model: "model_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 42)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 42)        │         85 │ input_layer_1[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      5,504 │ normalization_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     16,512 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     16,512 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128)       │          0 │ dense[0][0],      │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 128)       │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ re_lu[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     16,512 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     16,512 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128)       │          0 │ batch_normalizat… │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 128)       │          0 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ re_lu_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     16,512 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_5[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     16,512 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128)       │          0 │ batch_normalizat… │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 128)       │          0 │ add_2[0][0]     

 Total params: 210,006 (820.34 KB)

 Trainable params: 206,849 (808.00 KB)

 Non-trainable params: 3,157 (12.34 KB)

In [10]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [9]:
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_prc',
#     verbose=1,
#     patience=10,
#     mode='max',
#     restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
history = model.fit(
    ds_train,
    epochs=EPOCHS,
    # class_weight=class_weight,
    validation_data=ds_val,
    callbacks=[tensorboard_callback])

Epoch 1/100


2024-07-11 21:36:59.163978: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 849527280 exceeds 10% of free system memory.
I0000 00:00:1720708626.592639   56052 service.cc:145] XLA service 0x7fed0802db90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720708626.592729   56052 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-07-11 21:37:06.774822: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-11 21:37:07.474176: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720708628.972388   56480 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_4273', 164 bytes spill stores, 164 bytes spill loads

I0000 00:00:1720708629.419717   56481 asm_compiler.cc:

   5/9877 ━━━━━━━━━━━━━━━━━━━━ 2:24 15ms/step - accuracy: 0.4838 - auc: 0.4315 - fn: 3.4000 - fp: 392.4000 - loss: 0.3175 - prc: 0.0210 - precision: 0.0038 - recall: 0.3014 - tn: 370.4000 - tp: 1.8000               

I0000 00:00:1720708632.312659   56052 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_3', 40 bytes spill stores, 40 bytes spill loads

I0000 00:00:1720708632.318836   56052 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


9873/9877 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9818 - auc: 0.9423 - fn: 11416.7598 - fp: 5065.8130 - loss: 0.0149 - prc: 0.2603 - precision: 0.3550 - recall: 0.2139 - tn: 1243534.6250 - tp: 3854.8311

I0000 00:00:1720708757.566263   57425 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_4273', 128 bytes spill stores, 128 bytes spill loads

I0000 00:00:1720708758.586197   57417 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1159', 48 bytes spill stores, 48 bytes spill loads



9877/9877 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9818 - auc: 0.9423 - fn: 11420.3711 - fp: 5067.0552 - loss: 0.0149 - prc: 0.2604 - precision: 0.3550 - recall: 0.2140 - tn: 1244039.1250 - tp: 3857.4009

I0000 00:00:1720708760.795899   56052 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_3', 40 bytes spill stores, 40 bytes spill loads

I0000 00:00:1720708766.847414   57618 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_321', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1720708767.089837   57624 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_321', 40 bytes spill stores, 40 bytes spill loads

I0000 00:00:1720708767.090002   57616 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_321', 16 bytes spill stores, 32 bytes spill loads



9877/9877 ━━━━━━━━━━━━━━━━━━━━ 149s 14ms/step - accuracy: 0.9818 - auc: 0.9423 - fn: 11421.2734 - fp: 5067.3652 - loss: 0.0149 - prc: 0.2604 - precision: 0.3551 - recall: 0.2140 - tn: 1244165.2500 - tp: 3858.0432 - val_accuracy: 0.9929 - val_auc: 0.9936 - val_fn: 559.0000 - val_fp: 333.0000 - val_loss: 0.0052 - val_prc: 0.7472 - val_precision: 0.7440 - val_recall: 0.6339 - val_tn: 124558.0000 - val_tp: 968.0000
Epoch 2/100
9877/9877 ━━━━━━━━━━━━━━━━━━━━ 117s 12ms/step - accuracy: 0.9927 - auc: 0.9947 - fn: 6040.1514 - fp: 2956.0730 - loss: 0.0049 - prc: 0.7532 - precision: 0.7494 - recall: 0.5909 - tn: 1246276.5000 - tp: 9239.1650 - val_accuracy: 0.9945 - val_auc: 0.9973 - val_fn: 385.0000 - val_fp: 314.0000 - val_loss: 0.0037 - val_prc: 0.8535 - val_precision: 0.7843 - val_recall: 0.7479 - val_tn: 124577.0000 - val_tp: 1142.0000
Epoch 3/100
9877/9877 ━━━━━━━━━━━━━━━━━━━━ 118s 12ms/step - accuracy: 0.9946 - auc: 0.9974 - fn: 4248.1440 - fp: 2364.3608 - loss: 0.0036 - prc: 0.8580 - prec

In [ ]:
model.save("model_4.keras")
model.save("model_4_sm")